In [1]:
import numpy as np
from scipy import constants as const
from scipy import integrate as integ
from scipy import optimize as opt
import sys
import matplotlib.pyplot as plt

In [2]:
c = const.physical_constants['speed of light in vacuum'][0]*100
thomson = const.physical_constants['Thomson cross section'][0]*10000
thomson

6.6524587158e-25

In [6]:
def f(x):
    return (1.0-4.0/x-8.0/(x*x))*np.log(1.0+x) + 0.5 + 8.0/x - 0.5/((1.0+x)*(1.0+x))

def rate(om,g):
    beta = np.sqrt(1.0-1.0/(g*g))
    y = g*om
    factor = 1.0e3
    if beta < 1.0/factor and om < 1.0/factor:
        return c*thomson*(1.0-2.0*y)
    else:
        if om < 1.0/factor / (2.0*y*(1.0+beta)):
            return c*thomson*(1.0-2.0*y/3.0*(3.0+beta*beta))
        else:
            if om > factor*g/4.0 and g > factor:
                return 3.0*c*thomson/(8.0*y) * np.log(4.0*y)
            else:
                if g > 2.0 and g < 30.0 and om > 0.01 and om < 30.0:
                    return 3.0*c*thomson/(8.0*y) * ( (1.0-2.0/y - 2.0/(y*y))*np.log(1.0+2.0*y)+0.5 \
                                                   + 4.0/y - 0.5/(1.0+2.0*y)**2)
                else:
                    cte = 3.0*c*thomson/(32.0*y*y*beta)
                    min = 2.0*y*(1.0-beta)
                    max = 2.0*y*(1.0+beta)
                    return cte * integ.quad(f,min,max,epsabs=1.0e-3,epsrel=1.0e-3)[0]

def rateexact(om,g):
    beta = np.sqrt(1.0-1.0/(g*g))
    y = g*om
    cte = 3.0*c*thomson/(32.0*y*y*beta)
    min = 2.0*y*(1.0-beta)
    max = 2.0*y*(1.0+beta)
    return cte * integ.quad(f,min,max,epsabs=1.0e-3,epsrel=1.0e-3)[0]
                
def dPdz(z,om,omp,g):
    
    g2 = g*g
    beta = np.sqrt(1.0-1.0/g2)
    #z = (1.0-zeda)/beta
    zeda = 1.0-beta*z
    eps = omp/g
    rho = om/omp
    k = g/om
    k2 = k*k
    aux = rho*(beta*beta+eps*eps+2.0*beta*eps*z)
    y0 = (eps+beta*z)*(rho+eps*rho-zeda)/aux
    aux1 = rho*rho*beta*beta
    aux2 = 2.0*rho*eps*(1.0-rho)*zeda
    aux3 = (rho-zeda)*(rho-zeda)
    
    sqrt1 = aux1+aux2-aux3
    
    if sqrt1 < 0:
        print('rho = ',rho,'sqrt1 = ', sqrt1, 'zeda =', zeda)
        sys.exit()
    
    delta = beta*np.sqrt(1.0-z*z)*np.sqrt(sqrt1)/aux
    a = zeda-(1.0-y0)/k
    a2 = a*a
    b = delta/k
    b2 = b*b
    
    num = 3.0*thomson*c*om
    r = rateexact(omp,g)
    a2b2 = a2-b2
    if(a2b2 < 0.0):
        print(a2b2)
        sys.exit()
    
    minussqrta2b2 = 1.0/np.sqrt(a2b2)
    
    den = 16.0*g2*g2*omp*omp*r*np.sqrt(beta*beta+eps*eps+2.0*beta*eps*z)*zeda
    
    factor = 2*y0*k - a*k2 + minussqrta2b2*(1.0+y0*y0-2.0*a*y0*k+a2*k2) + 1.0/(k2*zeda) * \
    (k2 + k2*minussqrta2b2*a*(2*b-a)/(a-b)+np.power(minussqrta2b2,3)*(a*(1.0-y0)*(1.0-y0)+2.0*k* \
    b2*(1-y0)-b*a2*k2))
        
    result = num*factor/den
    
    return result
    
def prob(om,omp,g):
    beta = np.sqrt(1.0-1.0/(g*g))
    eps = omp/g
    #zedaminimum = zedamin(rho,eps,g)
    #zedamaximum = zedamax(rho,eps,g)
    zminimum = zmin(om,eps,g)
    zmaximum = zmax(om,eps,g)
    
    result = integ.quad(dPdz,zminimum,zmaximum,args=(om,omp,g),epsabs=1.0e-2,epsrel=1.0e-2)

    return result[0]

def auxprob(logom,omp,g):
    return prob(np.exp(logom),omp,g) * np.exp(logom)

In [4]:
def zedaminus(rho,eps,g):
    beta = np.sqrt(1.0-1.0/(g*g))
    d = 1.0+eps-rho*eps
    aux = d*d-1.0/(g*g)
    return rho*(d-np.sqrt(aux))
    
def zedaplus(rho,eps,g):
    beta = np.sqrt(1.0-1.0/(g*g))
    d = 1.0+eps-rho*eps
    aux = d*d-1.0/(g*g)
    return rho*(d+np.sqrt(aux))

def extrinf(rho,eps,g):
    beta = np.sqrt(1.0-1.0/(g*g))
    return zedaplus(rho,eps,g) - (1.0-beta)

def extrsup(rho,eps,g):
    return zedaminus(rho,eps,g) - (1.0+beta)

def zedamin(rho,eps,g):
    beta = np.sqrt(1.0-1.0/(g*g))
    return np.maximum(1.0-beta,zedaminus(rho,eps,g))

def zedamax(rho,eps,g):
    beta = np.sqrt(1.0-1.0/(g*g))
    return np.minimum(1.0+beta,zedaplus(rho,eps,g))

def zminus(rho,eps,g):
    beta = np.sqrt(1.0-1.0/(g*g))
    d = 1.0+eps-eps*rho
    aux = d*d-1.0/(g*g)
    if (aux < 0.0):
        aux = 0.0
    return (1.0-rho*(d-np.sqrt(aux)))/beta

def zplus(rho,eps,g):
    beta = np.sqrt(1.0-1.0/(g*g))
    d = 1.0+eps-eps*rho
    aux = d*d-1.0/(g*g)
    if (aux < 0.0):
        aux = 0.0
    return (1.0-rho*(d+np.sqrt(aux)))/beta    
    
def zmin(om,eps,g):
    omp = g*eps
    rho = om/omp
    return np.maximum(-1.0,zplus(rho,eps,g))

def zmax(om,eps,g):
    omp = g*eps
    rho = om/omp
    return np.minimum(1.0,zminus(rho,eps,g))

In [5]:
rhomaxAbs = 1.0 + (g-1.0)/omp
rhomax = ommaxAnalitic/omp
rhomin = omminNum/omp

rhoarray = np.arange(0.0001,rhomaxAbs,rhomaxAbs/100000.0)
plt.plot(rhoarray,zedaminus(rhoarray,eps,g),color='b')
plt.plot(rhoarray,zedaplus(rhoarray,eps,g),color='b')
plt.axhline(y=1+beta,color='r',linestyle=':')
plt.axhline(y=1-beta,color='r',linestyle=':')
plt.axvline(x=rhomax,color='r',linestyle=':')
plt.axvline(x=rhomin,color='r',linestyle=':')

rhonew = np.arange(rhomin,rhomax,rhomax/1000)
plt.fill_between(rhonew,zedamin(rhonew,eps,g),zedamax(rhonew,eps,g))

NameError: name 'g' is not defined

In [8]:
g = 1.1
beta = np.sqrt(1.0-1.0/(g*g))
zero = 1.0e-5


for omp in np.logspace(-12,5,20):
    
    eps = omp/g
    
    omminAnalitic = omp*(1.0-beta)/(1.0+2.0*beta*omp/g)
    omminNum = omp*opt.bisect(extrinf,0.0,1.0,args=(eps,g))

    ommaxAbs = omp + (g - 1.0)
    ommaxAnalitic = omp*(1+beta)/(1-beta+2.0*omp/g)
    #ommaxNum = omp*opt.bisect(extrsup,1.0,ommaxAbs/omp,args=(eps,g))

    if eps < beta/(1.0+g*(1.0+beta)):
        ommax = ommaxAnalitic
        print('Caso (a)')
    elif beta/(1.0+g*(1.0+beta)) < eps and eps < beta:
        ommax = ommaxAbs
        print('Caso (b)')
    elif beta < eps:
        ommax = ommaxAbs
        print('Caso (c)')

    min = np.log(omminNum)
    max = np.log(ommax)
    
    result = integ.quad(auxprob,min,max,args=(omp,g),epsabs=1.0e-2,epsrel=1.0e-2)
    print(omminNum,ommax,omp,result[0])

Caso (a)


/usr/lib/python3/dist-packages/scipy/integrate/quadpack.py:364: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  warnings.warn(msg, IntegrationWarning)
/usr/lib/python3/dist-packages/scipy/integrate/quadpack.py:364: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  warnings.warn(msg, IntegrationWarning)


4.11833347111e-13 2.42816665288e-12 1e-12 1.6126274599138615e-13
Caso (a)
3.23190325265e-12 1.90552799048e-11 7.84759970351e-12 2.9929659570647375e-13
Caso (a)
2.53626830055e-11 1.49538208907e-10 6.15848211066e-11 2.176648037126403e-08
Caso (a)


KeyboardInterrupt: 

In [25]:
g = 10.0
omp = 0.0199526

probtotexact = 1.49684e+06

eps = omp/g

omminNum = omp*opt.bisect(extrinf,0.0,1.0,args=(eps,g))
ommaxNum = omp*opt.bisect(extrsup,1.0,ommaxAbs/omp,args=(eps,g))

ommaxAbs = omp + (g - 1.0)
ommaxAnalitic = omp*(1+beta)/(1-beta+2.0*omp/g)

if eps < beta/(1.0+g*(1.0+beta)):
    ommax = ommaxNum
    print('Caso (a)')
elif beta/(1.0+g*(1.0+beta)) < eps and eps < beta:
    ommax = ommaxAbs
    print('Caso (b)')
elif beta < eps:
    ommax = ommaxAbs
    print('Caso (c)')
      
zero = 1.0e-5
ommax = ommax*(1.0-zero)
ommin = omminNum*(1.0+zero)

result = integ.quad(prob,ommin,ommax,args=(omp,g),epsrel=1.0e-10)
result[0]

ValueError: f(a) and f(b) must have different signs